<a href="https://colab.research.google.com/github/rahiakela/transfer-learning-for-natural-language-processing/blob/main/2-getting-started-with-baselines/2_linear_and_tree_based_models_for_movie_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear & Tree-based models for Movie Sentiment Classification

Our goal is to establish a set of baselines for a pair of concrete NLP problems, which we will later be able to use to measure progressive improvements gained from leveraging increasingly sophisticated transfer learning
approaches. In the process of doing this, we aim to advance your general NLP instincts and refresh your understanding of typical procedures involved in setting up problem-solving pipelines for such problems. You will review techniques ranging from tokenization to data structure and model selection. We first train some traditional machine learning models from scratch to establish some preliminary baselines for these problems.

We will focus on a pair of important representative example NLP problems – spam
classification of email, and sentiment classification of movie reviews. This exercise will arm you with a number of important skills, including some tips for obtaining, visualizing and preprocessing data. 

Three major model classes will be covered, namely linear models such as logistic regression, decision-tree-based models such as random forests, and neural-network-based models such as ELMo. These classes are additionally represented by support vector machines (SVMs) with linear kernels, gradient-boosting machines (GBMs) and BERT respectively. 

<img src='https://github.com/rahiakela/img-repo/blob/master/transfer-learning-for-natural-language-processing/content-classification-supervised-models.png?raw=1' width='800'/>



## Setup

In [1]:
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import email        # email package for processing email messages
import random
import re
import os
import time

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC                              # Support Vector Classification model
from sklearn.ensemble import RandomForestClassifier      # random forest classifier library
from sklearn.ensemble import GradientBoostingClassifier  # GBM algorithm

from sklearn.model_selection import GridSearchCV, cross_val_score         # for tune parameters systematically
from sklearn import metrics                              #Additional scklearn functions
from sklearn.metrics import accuracy_score

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
%%shell

wget -q "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
tar xzf aclImdb_v1.tar.gz

rm -rf aclImdb_v1.tar.gz
rm -rf aclImdb/train/unsup

## Preprocessing Movie Sentiment Classification Example Data

This notebook is concerned with classifying movie reviews from IMDB into
positive or negative sentiments expressed. This is a prototypical sentiment analysis example that has been used widely in the literature to study many algorithms.

We will use a popular labeled dataset of 25000 reviews for this, which was assembled by scraping the popular movie review website IMDB and mapping the number of stars corresponding to each review to either 0 or 1 – depending on whether it was less than or greater than 5 out of 10 stars respectively.It has been used widely in prior NLP literature, and this familiarity is part of the reason we choose it as an illustrative example for baselining.

The sequence of steps used to preprocess each IMDB movie review before analysis is very similar to the one presented for the email spam classification example.

The first major difference is that no email headers are attached to these reviews, so the header extraction step is not applicable. 

Additionally, since some stopwords – including “no” and “not” – may
change the sentiment of the message, the stopword removal step may need to be carried out with extra care, first making sure to drop such stopwords from the target list. We did experiment with dropping such words from the list, and saw little to no effect on the result. This is likely because other non-stopwords in the reviews are very predictive features, rendering this step irrelevant.

<img src='https://github.com/rahiakela/img-repo/blob/master/transfer-learning-for-natural-language-processing/spam-email-preprocessing.png?raw=1' width='800'/>


### IMDB Movie Review Dataset preprocessing

Before proceeding, we must decide how many samples to draw from each class. We must also decide the maximum number of tokens per email, and the maximum length of each token. This is done by setting the following overarching hyperparameters.

In [4]:
n_sample = 1000   # number of samples to generate in each class
maxtokens = 50    # the maximum number of tokens per document
maxtokenlen = 20  # the maximum length of each token

With these hyperparameters specified, we can now create a single DataFrame for the overarching training dataset. Let’s take the opportunity to also perform remaining preprocessing tasks, namely removing stop words, punctuations and tokenizing.

#### Tokenization

Let’s proceed by defining a function to tokenize text by splitting them into 
words.

In [5]:
def tokenize(row):
  if row is None or row is "":
    tokens = ""
  else:
    tokens = row.split(" ")[:maxtokens]
  return tokens

#### Remove punctuation and unnecessary characters

**In order to ensure that classification is done based on language content only, we have to remove punctuation marks and other non-word characters from the emails.** We do this by employing regular expressions with the Python regex library. We also normalize words by turning them into lower case.

In [6]:
def reg_expressions(row):
  tokens = []
  try:
    for token in row:
      token = token.lower()          # make all characters lower case
      token = re.sub(r"[\W\d]", "", token)
      token = token[:maxtokenlen]    # truncate all tokens to hyperparameter maxtokenlen
      tokens.append(token)
  except:
    token = ""
    tokens.append(token)
  return tokens

#### Stop-word removal

Stop-words are also removed. Stop-words are words that are very common in text but offer no useful information that can be used to classify the text. Words such as is, and, the, are are examples of stop-words. The NLTK library contains a list of 127 English stop-words and can be used to filter our tokenized strings.

In [7]:
stop_words = stopwords.words("english")
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
# it may be beneficial to drop negation words from the removal list, as they can change the positive/negative meaning of a sentence
stop_words.remove("no")
stop_words.remove("nor")
stop_words.remove("not")

In [9]:
def stop_word_removal(row):
  token = [token for token in row if token not in stop_words]
  token = filter(None, token)

  return token

### Converting the Sentiment Text Into Numbers

We start by employing what is often considered the simplest method for vectorizing words, i.e., converting them into numerical vectors – the bag-of-words model. This model simply counts the frequency of word tokens contained in each email and thereby represents it as a vector of such frequency counts.

Please observe that in doing this, we only retain tokens that appear more than once, as captured by the variable “used_tokens”. This enables us to keep the vector dimensions significantly lower than they would be otherwise. Please also
note that one can achieve this using various in-built vectorizers in the popular library scikitlearn.

We also note the scikit-learn vectorization methods include counting occurrences of sequences of any n words, or n-grams, as well as the tf-idf approach – important fundamental concepts you should brush on if rusty. For the problems looked at here, we did not notice an improvement when using these vectorization methods over the bag-of words approach.

For the computer to make inferences of the sentiment, it has to be able to interpret the text by making a numerical representation of it. One way to do this is by using something called a "bag-of-words" model. This model simply counts the frequency of word tokens for each sentiment and thereby represents it as a vector of these counts.

The assemble_bag() function assembles a new dataframe containing all the unique words found in the text documents. It counts the word frequency and then returns the new dataframe.

In [10]:
def assemble_bag(data):
  used_tokens = []
  all_tokens = []

  for item in data:
    for token in item:
      if token in all_tokens:
        # If token has been seen before, append it to output list used_tokens
        if token not in used_tokens:
          used_tokens.append(token)
      else:
        all_tokens.append(token)

  df = pd.DataFrame(0, index=np.arange(len(data)), columns=used_tokens)

  # Create a Pandas DataFrame counting frequencies of vocabulary words – corresponding to columns, in each email – corresponding to rows
  for i, item in enumerate(data):
    for token in item:
      if token in used_tokens:
        df.iloc[i][token] += 1

  return df

Having fully vectorized the dataset, we must remember that it is not shuffled with respect to classes, i.e., it contains Nsamp = 1000 spam emails followed by an equal number of nonspam emails. Depending on how this dataset is split, in our case by picking the first 70% for training and the remainder for testing, this could lead to a training set composed of spam only, which would obviously lead to failure. In order to create a randomized ordering of class samples in the dataset, we will need to shuffle the data in unison with the header/list of labels.

In [21]:
# shuffle raw data first
def unison_shuffle_data(data, header):
  p = np.random.permutation(len(header))
  data = data[p]
  header = np.asarray(header)[p]

  return data, header

Let's load dataset into a Numpy array after tokenizing, removing stopwords and punctuations, and shuffling.

In [22]:
# load data in appropriate form
def load_data(path):
  data, sentiments = [], []
  for folder, sentiment in (("neg", 0), ("pos", 1)):
    folder = os.path.join(path, folder)
    for name in os.listdir(folder):
      with open(os.path.join(folder, name), "r") as reader:
        text = reader.read()
      text = tokenize(text)            # tokenizing
      text = stop_word_removal(text)   # removing stopwords and punctuations
      text = reg_expressions(text)
      data.append(text)
      sentiments.append(sentiment)     # Track corresponding sentiment labels

  # converting to Numpy array
  data_np = np.array(data)
  # shuffling
  data, sentiments = unison_shuffle_data(data_np, sentiments)

  return data, sentiments

In [23]:
train_path = os.path.join("aclImdb", "train")
test_path = os.path.join("aclImdb", "test")

raw_data, raw_header = load_data(train_path)
print(raw_data.shape)
print(len(raw_header))

(25000,)
25000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


Let's take n_sample*2 random entries of the loaded data for training:

In [ ]:
# Subsample required number of samples
random_indices = np.random.choice(range(len(raw_header)), size=(n_sample * 2, ), replace=False)
data_train = raw_data[random_indices]
header = raw_header[random_indices]

print("DEBUG::data_train::")
print(data_train[:5])

Before proceeding, we need to check the balance of the resulting data with regards to class. In general, we don’t want one of the labels to represent most of the dataset, unless that is the distribution expected in practice.

In [25]:
# Display sentiments and their frequencies in the dataset, to ensure it is roughly balanced between classes
unique_elements, counts_elements = np.unique(header, return_counts=True)
print("Sentiments and their frequencies:")
print(unique_elements)
print(counts_elements)

Sentiments and their frequencies:
[0 1]
[ 985 1015]


We are now ready to convert these into numerical vectors!!

Having satisfied ourselves that the data is roughly balanced between the two classes, with each class representing roughly half of the dataset, assemble and visualize the bag-of-words representation.

In [26]:
# create bag-of-words model
mixed_bag_reviews  = assemble_bag(data_train) 

# this is the list of words in our bag-of-words model
predictors = [column for column in mixed_bag_reviews.columns]

# expand default pandas display options to make emails more clearly visible when printed
pd.set_option("display.max_colwidth", 300)

mixed_bag_reviews

,i,the,good,jim,carrey,not,br,it,movie,film,task,time,thing,would,,rating,holes,hard,also,one,in,actors,could,its,many,find,another,little,mystery,romance,made,but,ditched,this,worse,audience,bad,hindi,seen,and,...,tour,akshay,kareena,kapoor,bikinibr,whoa,oregon,olympic,mercifully,gymnast,gymnastic,minus,gifted,hamiltons,siblings,eldest,surrogate,parent,unsure,hello,stole,pounds,hint,persuade,mutants,range,effectively,levels,technique,hardhitting,further,planning,cliches,rave,endeavor,filmsbr,adaptations,terrorist,grieves,deaths
0,2,0,1,0,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,2,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,1,4,4,1,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,3,2,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
1996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1997,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1998,3,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


With this numerical representation ready, we now proceed to building out our baseline classifiers in the subsequent sections for the two presented example datasets.

As the very last step of preparing the sentiment dataset for training by our baseline classifiers, we split it into independent training and testing or validation sets. This will allow us to evaluate the performance of the classifier on a set of data that was not used for training, an important thing
to ensure in machine learning practice. We elect to use 70% of the data for training, and 30% for testing/validation afterwards.

In [27]:
# split into independent 70% training and 30% testing sets
data = mixed_bag_reviews.values

idx = int(0.7 * data.shape[0])  # get 70% index value

# 70% of data for training
train_x = data[:idx, :]
train_y = header[:idx]

# remaining 30% for testing
test_x = data[idx:, :]
test_y = header[idx:]

print("train_x/train_y list details, to make sure they are of the right form:")
print(len(train_x))
print(train_x)
print(len(train_y))
print(train_y[:5])

train_x/train_y list details, to make sure they are of the right form:
1400
[[2 0 1 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [2 2 0 ... 0 0 0]]
1400
[0 0 1 1 0]


Since 70% of 2000 is 1400, looks good! (for n_sample=1000)

### How about other vectorization strategies?

Let's check some other vectorization strategies.

In [29]:
# create the transform - uncomment the one you want to focus on
# vectorizer = CountVectorizer()    # this is equivalent to the bag of words
vectorizer = TfidfVectorizer()    # tf-idf vectorizer
# vectorizer = HashingVectorizer(n_features=3000)  # hashing vectorizer

In [31]:
# build vocabulary
vectorizer.fit([" ".join(sublist) for sublist in data_train])
# summarize
print(len(vectorizer.vocabulary_))
print(vectorizer.idf_)

# encode one document
vector = vectorizer.transform([" ".join(data_train[0])])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

# set this to 'True' if you want to use the vectorizer featurizers instead of the bag-of-words done before
USE = True
if USE:
  data = vectorizer.transform([" ".join(sublist) for sublist in data_train]).toarray()
  # 70% of data for training
  train_x = data[:idx, :]
  # remaining 30% for testing
  test_x = data[idx, :]

  print("train_x/train_y list details, to make sure it is of the right form:")
  print(train_x.shape[0])
  print(train_x)
  print(train_y[:5])
  print(len(train_y))
  predictors = [column for column in vectorizer.vocabulary_]

12095
[7.90825515 7.90825515 7.90825515 ... 7.90825515 7.50279005 7.90825515]
(1, 12095)
[[0. 0. 0. ... 0. 0. 0.]]
train_x/train_y list details, to make sure it is of the right form:
1400
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[0 0 1 1 0]
1400


## Generalized Linear Models